# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'

df = pd.read_csv(url)

df

df.rename(columns= {
                    'Customer': 'customer', 
                    'State' : 'state', 
                    'Gender' : 'gender',
                    'Education' : 'education',
                    'Customer Lifetime Value' : 'customer_lifetime_value',
                    'Response': 'response',
                    'Coverage': 'coverage',
                    'Effective To Date' : 'effective_to_date',
                    'EmploymentStatus' : 'employment_status',
                    'Number of Open Complaints' : 'number_of_open_complaints',
                    'Number of Policies' : 'number_of_policies',
                    'Policy Type' : 'policy_type',
                    'Policy' : 'policy',
                    'Renew Offer Type' : 'renew_offer_type',
                    'Sales Channel' : 'sales_channel',
                    'Vehicle Size' : 'vehicle_size',
                    'Vehicle Type' : 'vehicle_type',
                    'Vehicle Class' : 'vehicle_class',
                    'Total Claim Amount' : 'total_claim_amount'}, inplace=True)



df['response'].unique()
df.info()


grouped_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]
display(grouped_df)


grouped_df.pivot_table(index='gender', columns='policy_type', values='total_claim_amount', aggfunc='sum')
#df(df['total_claim_amount'] > 1000) & (df['policy_type']) & (df['gender']) & (df['response'] == 'Yes')]   original code which worked



filtered_df= df.groupby(by=['state','policy']).agg({'state': 'count'})
display(filtered_df)

filtered_df[filtered_df > 500]

grouped_counts = df.groupby(['state', 'policy_type']).size().reset_index(name='customer_count')

# Filter to only include groups with more than 500 customers
filtered_counts = grouped_counts[grouped_counts['customer_count'] > 500]

# Print the results
print(filtered_counts)


df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).round(2)


df['month'] = df['date_of_purchase'].dt.strftime('%Y-%m')

# Create pivot table
pivot_table = df.pivot_table(index='state', columns='month', values='policy_id', aggfunc='count', fill_value=0)

# Print the results
print(pivot_table)

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here